In [2]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from dt_apriltags import Detector
import matplotlib.cm as cm
from lane_detection import *
from lane_following import *

In [3]:
def drawLines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 90, 100, apertureSize=3) 
    lines = cv2.HoughLinesP(
                    edges, #described above
                    1, #1 pixel resolution parameter
                    np.pi/180, # 1 degree resolution parameter
                    10, #min number of intersections/votes
                    minLineLength=10,
                    maxLineGap=60,
            ) # detect lines
    try:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            slope = (y2-y1)/(x2-x1)
            print(str(slope))
    except TypeError:
        pass
    
    return img


In [ ]:
img = cv2.imread('rov_pool.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
edges = cv2.Canny(gray, 106, 107, apertureSize=3) # detect edges, gray is image in grayscale, 50 and 150 represent 2 images that have been threshholded at 2 different levels, apertureSize controls how much light the image gets and how exposed it is
lines = cv2.HoughLinesP(
                edges, #described above
                1, #1 pixel resolution parameter
                np.pi/180, # 1 degree resolution parameter
                60, #min number of intersections/votes
                minLineLength=475,
                maxLineGap=100,
        ) # detect lines

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    slope = (y2-y1)/(x2-x1)
    print(str(slope))

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [5]:
import cv2
import numpy as np

video = cv2.VideoCapture('AUV_Vid.mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret, frame = video.read()
count = 0
frequency = 1

while ret:
    if count % frequency == 0:
        processed_frame = drawLines(frame)
        output_video.write(processed_frame)

    count += 1
    ret, frame = video.read()

video.release()
output_video.release()

1920
1080
0.423963133640553
0.4233128834355828
1.9603174603174602
0.421259842519685
1.9666666666666666
4.8125
0.36
1.96875
4.416666666666667
1.9609375
0.4246575342465753
0.3333333333333333
0.35294117647058826
0.4219269102990033
1.9649122807017543
0.42441860465116277
1.9509803921568627
1.9646017699115044
0.36470588235294116
4.25
0.3829787234042553
0.4444444444444444
0.34615384615384615
5.0625
0.4444444444444444
0.3695652173913043
0.42592592592592593
1.9696969696969697
0.42316258351893093
1.9649122807017543
1.971698113207547
0.4234527687296417
0.36401673640167365
1.9610389610389611
0.421875
4.8
0.3626373626373626
1.9649122807017543
4.166666666666667
1.9639639639639639
0.4224137931034483
1.9519230769230769
0.3656957928802589
0.4198895027624309
1.9642857142857142
1.9651162790697674
0.43478260869565216
4.2
0.4051724137931034
4.416666666666667
1.9672131147540983
0.4066666666666667
1.9540229885057472
0.40875912408759124
0.34615384615384615
1.9534883720930232
1.9639639639639639
0.3589743589743

/tmp/ipykernel_3856/470011640.py:16: RuntimeWarning: divide by zero encountered in scalar divide
  slope = (y2-y1)/(x2-x1)


2.3516483516483517
2.358974358974359
6.2631578947368425
6.176470588235294
2.343283582089552
2.378787878787879
2.34375
6.352941176470588
2.260869565217391
2.257425742574257
2.2421052631578946
6.346153846153846
2.2545454545454544
2.2555555555555555
6.411764705882353
8.0
2.357142857142857
2.36
2.3417721518987342
6.407407407407407
6.269230769230769
2.3544303797468356
6.3125
2.36
2.263157894736842
8.133333333333333
2.276595744680851
2.2653061224489797
2.235294117647059
6.666666666666667
7.222222222222222
7.75
2.2758620689655173
2.2526315789473683
2.3488372093023258
2.3620689655172415
7.95
2.3626373626373627
8.166666666666666
7.208333333333333
7.909090909090909
2.2551020408163267
8.105263157894736
2.257142857142857
10.125
2.255813953488372
8.5
2.269230769230769
2.6470588235294117
2.260869565217391
8.133333333333333
8.333333333333334
2.360544217687075
2.3564356435643563
2.3524590163934427
2.346774193548387
7.1891891891891895
7.25
7.32
7.148148148148148
2.3515625
2.357142857142857
8.2083333333

KeyboardInterrupt: 

In [ ]:
from dt_apriltags import Detector
import cv2
import numpy as np


cameraMatrix = np.array([ 1060.71, 0, 960, 0, 1060.71, 540, 0, 0, 1]).reshape((3,3))

camera_params = ( cameraMatrix[0,0], cameraMatrix[1,1], cameraMatrix[0,2], cameraMatrix[1,2] )


video = cv2.VideoCapture('AprilTagTest.mkv')

fps = int(video.get(cv2.CAP_PROP_FPS))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_file = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_file, fourcc, 30, (width, height))

ret, frame = video.read()
count = 0
frequency = 1


while ret:
    if count % frequency == 0:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        at_detector = Detector(families='tag36h11', #controls what it is supposed to detect
                       nthreads=1, #controls the number of threads used in the detection
                       quad_decimate=1.0, #controls to what scale the image is lowered in resolution
                       quad_sigma=0.0, #blur applied to the image to reduce noise
                       refine_edges=1, #attempts to redefine the edges of the tag boundaries
                       decode_sharpening=0.25, #controls how much the image is sharpened after the tags are discovered
                       debug=0) # no debugging information will be produced

        tags = at_detector.detect(frame, estimate_tag_pose=True, camera_params=camera_params, tag_size=0.1)

        color_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

        for tag in tags:
            for idx in range(len(tag.corners)):
                cv2.line(color_frame, tuple(tag.corners[idx - 1, :].astype(int)), tuple(tag.corners[idx, :].astype(int)), (0, 255, 0))

            cv2.putText(color_frame, str(tag.tag_id),
                        org=(tag.corners[0, 0].astype(int) + 10, tag.corners[0, 1].astype(int) + 10),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.8,
                        color=(0, 0, 255))
            print(tag.pose_t)
            print(tag.pose_R)

        output_video.write(color_frame)

    count += 1
    ret, frame = video.read()

video.release()
output_video.release()

In [ ]:
cap = cv2.VideoCapture('AUV_Vid.mkv')
img_array = []
i = 0
ret = True
detectedLane = 0
failedToDetectLane = 0
while ret:
    if i > 1300:
        break
    slopeSet = set() 
    i += 1
    ret, frame = cap.read()
    if(i%1 == 0 and i > 800):
        lines = detect_lines(frame, 30, 90, 3,10,10)
    
        try:
            lanes = detect_lanes(lines)
            pickedLane = pick_lane(lanes)
            frame = draw_Single_lane(frame, pickedLane, (255, 0, 0))
            frame = draw_lines(frame, lines,(0, 255, 0))
            center_intercept, center_slope = get_lane_center(pickedLane)
            xPoint = pickedLane[0][2]
            yPoint = pickedLane[0][3]
            cv2.line(frame, (int(center_intercept), 1080), (int(xPoint), int(yPoint)), (0,0,255), 3)
            rec_dir = recommend_direction(center_intercept, center_slope)
            print (rec_dir)
            cv2.putText(frame, rec_dir, (0, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            detectedLane += 1
        except:
            failedToDetectLane += 1
            pass
        
        img_array.append(frame)
        height, width, layers = frame.shape
        size = (width,height)
    
        accuracy = (detectedLane/(detectedLane+failedToDetectLane))*100
        cv2.putText(frame, str(accuracy), (0, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        print(f"accuracy: {accuracy}%")
        print(f"currently on frame: {i}")

out = cv2.VideoWriter('testVid8.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()